In [1]:
import pandas as pd

In [2]:
surveys = pd.read_csv('data/JobsData_Clean.csv', low_memory=False)
people = pd.read_csv('data/ContactsProcessed.csv', encoding='latin-1')

In [4]:
surveys.Emp_Stat_Type.value_counts()

Emp_Stat_Type
At Grad Placement Status Check          5680
3-Month Placement Status Check          3032
3-Month Post-Placement Status Check     2154
6-Month Post-Placement Status Check     1375
6-Month Placement Status Check          1165
9-Month Placement Status Check           748
9-Month Post-Placement Status Check      744
12-Month Placement Status Check          104
12-Month Post-Placement Status Check     103
Name: count, dtype: int64

We want to "6 month retention" in the sense of "of people who got a job out of EFE, how many kept that job for at least six months?"

So for us to know, we need a "6-Month Post-Placement Status Check".

Well; really, we could use a 9 or 12-month, maybe, if they have that but not 6...

In [9]:
len(set(surveys.ContactID[surveys.Emp_Stat_Type == '9-Month Post-Placement Status Check'])
   -set(surveys.ContactID[surveys.Emp_Stat_Type == '6-Month Post-Placement Status Check']))

306

In [10]:
len(set(surveys.ContactID[surveys.Emp_Stat_Type == '12-Month Post-Placement Status Check'])
   -set(surveys.ContactID[surveys.Emp_Stat_Type == '6-Month Post-Placement Status Check']))

39

And if we just have a 3, then at least we know the person got a job...

We should probably treat this like survival data?

In [11]:
(set(surveys.ContactID[surveys.Emp_Stat_Type == '12-Month Post-Placement Status Check'])
   -set(surveys.ContactID[surveys.Emp_Stat_Type == '6-Month Post-Placement Status Check']))

{19,
 53,
 141,
 143,
 151,
 158,
 163,
 171,
 176,
 234,
 271,
 413,
 443,
 501,
 515,
 831,
 842,
 888,
 949,
 986,
 1032,
 1251,
 1255,
 1260,
 1332,
 1351,
 1354,
 1495,
 1551,
 1606,
 1746,
 1753,
 1796,
 1920,
 1923,
 2200,
 2312,
 2417,
 5424}

Lots of dates are missing... `Employment_Start_Date`, `Employment_End_Date` are there sometimes... `Survey_Created_Date` maybe?

In [12]:
surveys.Survey_Created_Date.isna().sum()

188

Ah, these are all `Secured_Previous_Emp` rows though. Not real surveys?

Okay, so for each ContactID, go through their surveys in `Survey_Created_Date` order...

As soon as they have a non-U job number, try to get the start date, or use the survey date as the start date.

If you have one of these dates, we know you got a job. Otherwise, we don't know, since you could go on not getting a job for a long time but eventually get one.

From there we want to know whether you were employed _in that same job_ at six months.

So we keep reading further surveys. As long as they have the same jobid, we take that as knowledge that the person still has the job at such a date.

As soon as we have a survey with a different jobid, we use either that survey date or (preferably) the employement end date...

In [14]:
# Exclude surveys without dates
# (this drops some ContactIDs!)
dated_surveys = surveys[~surveys.Survey_Created_Date.isna()]

In [16]:
len(set(surveys.ContactID)) - len(set(dated_surveys.ContactID))

56

In [51]:
results = []

for person in sorted(set(dated_surveys.ContactID)):
    data = dated_surveys[dated_surveys.ContactID == person].copy()
    data = data.sort_values(by='Survey_Created_Date')
    first_jobid, first_start, first_end = None, None, None
    still_have = None
    six_mo_crude = None
    for index, row in data.iterrows():
        # I don't observe "U" rows that have prior employment start and end (so don't worry about that case)
        if row.Emp_Stat_Type == '6-Month Post-Placement Status Check':
            if first_jobid is None:
                six_mo_crude = not row.JobID.startswith('U')
            else:
                six_mo_crude = first_jobid == row.JobID
        if first_start is None and not row.JobID.startswith('U'):
            # got a new job!
            first_jobid = row.JobID
            first_start = row.Employment_Start_Date if not pd.isna(row.Employment_Start_Date) else row.Survey_Created_Date
            still_have = row.Survey_Created_Date
            # print(f'{first_jobid=}, {first_start=}, {still_have=}')
            # sometimes these also have an end date, but it must be for the prior job?
            continue
        if first_jobid is not None and row.JobID == first_jobid and first_end is None:
            # still have that job!
            still_have = row.Survey_Created_Date
            # print(f'{first_jobid=}, {first_start=}, {still_have=}')
            continue
        if first_jobid is not None and row.JobID != first_jobid and first_end is None:
            # new job or now unemployed!
            first_end = row.Employment_End_Date  # insist on this; survey date is too late
            # print(f'{first_jobid=}, {first_start=}, {still_have=}, {first_end=}')
    results.append([person, first_start, still_have, first_end, six_mo_crude])
    # print(data)
    # break

In [55]:
results = pd.DataFrame(results, columns=['ContactID', 'first_start', 'still_have', 'first_end', 'six_mo_crude'])

In [56]:
results.six_mo_crude.value_counts()

six_mo_crude
True     912
False    437
Name: count, dtype: int64

In [57]:
912 / (912 + 437)

0.676056338028169

In [58]:
912 + 437

1349

In [59]:
results

,ContactID,first_start,still_have,first_end,six_mo_crude
0,1,2021-12-01 00:00:00,2022-01-17 00:00:00,2021-12-01 00:00:00,False
1,2,None,None,None,None
2,3,2022-03-09 00:00:00,2022-03-09 00:00:00,None,None
3,4,2022-04-01 00:00:00,2022-04-21 00:00:00,None,None
4,5,2022-01-01 00:00:00,2022-07-13 00:00:00,None,None
...,...,...,...,...,...
7063,7120,2021-12-08 00:00:00,2022-03-29 00:00:00,None,None
7064,7121,2022-09-01 00:00:00,2022-09-20 00:00:00,None,None
7065,7122,2021-01-29 00:00:00,2021-10-18 00:00:00,None,True
7066,7123,2022-02-15 00:00:00,2022-07-21 00:00:00,None,None


In [60]:
(~results.first_start.isna()).sum()

5256

In [61]:
(~results.first_end.isna()).sum()

1052

In [68]:
results['survived_for'] = pd.to_datetime(results.still_have) - pd.to_datetime(results.first_start)

In [69]:
results['ended_in'] = pd.to_datetime(results.first_end) - pd.to_datetime(results.first_start)

In [71]:
results[~results.first_end.isna()]

,ContactID,first_start,still_have,first_end,six_mo_crude,survived_for,ended_in
0,1,2021-12-01 00:00:00,2022-01-17 00:00:00,2021-12-01 00:00:00,False,47 days,0 days
8,9,2021-07-05 00:00:00,2021-07-16 00:00:00,2022-04-04 00:00:00,None,11 days,273 days
16,18,2021-05-31 00:00:00,2022-02-25 00:00:00,2022-01-03 00:00:00,True,270 days,217 days
17,19,2021-04-01 00:00:00,2021-08-25 00:00:00,2021-07-21 00:00:00,None,146 days,111 days
34,36,2021-10-30 00:00:00,2021-12-28 00:00:00,2022-01-01 00:00:00,None,59 days,63 days
...,...,...,...,...,...,...,...
6579,6635,2022-09-05 00:00:00,2022-09-28 00:00:00,2022-06-01 00:00:00,None,23 days,-96 days
6585,6641,2022-09-05 00:00:00,2022-09-28 00:00:00,2022-06-14 00:00:00,None,23 days,-83 days
6589,6645,2022-09-05 00:00:00,2022-09-28 00:00:00,2022-06-16 00:00:00,None,23 days,-81 days
6592,6648,2022-08-30 00:00:00,2022-09-28 00:00:00,2022-06-08 00:00:00,None,29 days,-83 days


Yikes; this is bad... The end time seems very unreliable...

In [74]:
quick_export = results[~results.six_mo_crude.isna()][['ContactID', 'six_mo_crude']].copy()

In [75]:
quick_export

,ContactID,six_mo_crude
0,1,False
9,10,True
13,15,True
16,18,True
21,23,True
...,...,...
5665,5718,False
5672,5725,False
5674,5727,False
7060,7117,True


In [76]:
quick_export['six_month_retained'] = quick_export.six_mo_crude.astype(int)

In [77]:
quick_export = quick_export[['ContactID', 'six_month_retained']]

In [78]:
quick_export

,ContactID,six_month_retained
0,1,0
9,10,1
13,15,1
16,18,1
21,23,1
...,...,...
5665,5718,0
5672,5725,0
5674,5727,0
7060,7117,1


In [79]:
quick_export.to_csv('20230916-six_month_retained.csv', index=False)

How similar is that to the old way?

In [83]:
[column for column in list(people.columns) if '6' in column]

['pl_6',
 'pl_6_data',
 'X6.Month.Job.Retention',
 'X6.Month.Continuously.Working',
 'X6.Month.Post.Placement.Data.Avail.YES',
 'Retention_6_months',
 'Retention at 6 Months']

In [85]:
compare = pd.merge(quick_export, people[['ContactID', 'Retention_6_months']])

In [86]:
compare[compare.six_month_retained != compare.Retention_6_months]

,ContactID,six_month_retained,Retention_6_months
5,26,0,1.0
18,120,1,0.0
19,121,0,1.0
38,183,0,1.0
68,224,0,1.0
...,...,...,...
1232,4932,0,1.0
1272,5128,0,1.0
1275,5132,0,1.0
1285,5175,0,1.0


In [94]:
compare.old_way.isna().sum()

7

In [89]:
compare.columns = ['ContactID', 'new_way', 'old_way']

Aaron Schumacher worked on a new way of defining 6-month retention from the `JobsData_Clean.csv`.

There was some thought that the "old way" (`Retention_6_months` in `ContactsProcessed.csv`) was not ideal, possibly because it was not differentiating between retention in first jobs vs. working second (etc.) jobs, or possibly even other issues like just checking for employment at six months out from completing the program.

In [91]:
pd.crosstab(compare.new_way, compare.old_way)

old_way,0.0,1.0
new_way,,
0,315,94
1,27,869


Comparing the old way of calculating 6-month retention (0.0/1.0, across the top) with the new way of calculating 6-month retention by looking at '6-Month Post-Placement Status Check' surveys and checking that people are still working the _same_ job at that point, there are several comparisons.

For one thing, the new way gives us fewer labels to work with. There are only 1,349 results with the "new way," compared to 2,653 results with the "old way."

For the 1,305 people where we have both results, there is 91% agreement. The "new way" is somewhat more likely to say "no" where the "old way" said "yes," supporting the hypothesis that the "old way" sometimes marked people as 6-month retention where it shouldn't have.